In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import  plot_model


In [ ]:
try:
    import os
    from google.colab import drive

    drive.mount('/content/drive')
    print(os.getcwd())
    os.chdir("/content/drive/MyDrive/CorsoForecastingEnergeticoAvanzato")
    print(os.getcwd())
except:
    pass

In [ ]:
# def load_data(col=None, path="/kaggle/input/energy-consumption-generation-prices-and-weather/energy_dataset.csv", verbose=False):
def load_data(col=None, path="./data/SpainPrice/energy_dataset.csv", verbose=False):
    df = pd.read_csv(path)
    if col is not None:
        df = df[col]
    if verbose:
        print(df.head())
    return df

print("Multivariate Sample")
multivar_df = load_data(['time', 'price actual','total load actual'], verbose=True)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

df = load_data(col=["price day ahead","price actual"])
y_true = df.iloc[:,1]
y_pred_forecast = df.iloc[:,0]

baseline_rmse = mean_squared_error(y_true, y_pred_forecast,squared=False)

print(f"\nAverage RMSE in EUR/MWh for TSO Forecast ",baseline_rmse)

In [ ]:
df.iloc[:24*31].plot(figsize=(33,8))

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

df_load = load_data(col=["total load forecast","total load actual"]).dropna(axis=0)

y_true = df_load.iloc[:,1]
y_pred_forecast = df_load.iloc[:,0]

baseline_rmse = mean_squared_error(y_true,y_pred_forecast,squared=False)
print(f"\nAverage error in EUR/MWh for TSO Forecast ",baseline_rmse)

In [ ]:
df.iloc[:24*31].plot(figsize=(33,8))

In [ ]:
df.iloc[:24*31].plot(figsize=(33,8))

In [ ]:
def clean_data(series):
    """Fills missing values. 
    
        Interpolate missing values with a linear approximation.
    """
    series_filled = series.interpolate(method='linear')
        
    return series_filled
        
    
def min_max_scale(dataframe):
    """ Applies MinMax Scaling
    
        Wrapper for sklearn's MinMaxScaler class.
    """
    mm = MinMaxScaler((-1,1))
    return mm.fit_transform(dataframe)

In [ ]:
def make_datetime_features(series):
    
    #convert series to datetimes
    times = series.apply(lambda x: x.split('+')[0])
    datetimes = pd.DatetimeIndex(times)
    
    hours = datetimes.hour.values
    day = datetimes.dayofweek.values
    months = datetimes.month.values
    
    hour = pd.Series(hours, name='hours')
    dayofw = pd.Series(day, name='dayofw')
    month = pd.Series(months, name='months')
    
    return hour, dayofw, month


In [ ]:

hour, day, month = make_datetime_features(multivar_df.time)
print("Hours")
print(hour.head())
print("Day of Week")
print(day.head())
print("Months")
print(month.head())

In [ ]:
def split_data(series, train_fraq, test_len=24*265):
    """Splits input series into train, val and test.
    
        Default to 1 year of test data.
    """
    #slice the last year of data for testing 1 year has 8760 hours
    test_slice = len(series)-test_len

    test_data = series[test_slice:]
    train_val_data = series[:test_slice]
    
    #make train and validation from the remaining
    train_size = int(len(train_val_data) * train_fraq)
    val_size   = int(len(train_val_data) * (1-train_fraq))
    
    train_data = train_val_data[:train_size]
    val_data   = train_val_data 
    
    print("train_data.shape = ", train_data.shape)
    print("val_data.shape   = ", val_data.shape)
    print("test_data.shape  = ", test_data.shape)
    
    
    return train_data, train_val_data, test_data


In [ ]:
multivar_df = clean_data(multivar_df)

#add hour and month features
hours, day, months = make_datetime_features(multivar_df.time)
multivar_df = pd.concat([multivar_df.drop(['time'], axis=1), hours, day, months], axis=1)
multivar_df.head()

In [ ]:

#scale
multivar_df = min_max_scale(multivar_df)

train_multi, val_multi, test_multi = split_data(multivar_df, train_fraq=0.65, test_len=8760)


In [ ]:
print("Multivarate Datasets")
print(f"Train Data Shape: {train_multi.shape}")
print(f"Val Data Shape: {val_multi.shape}")
print(f"Test Data Shape: {test_multi.shape}")
print(f"Nulls In Train {np.any(np.isnan(train_multi))}")
print(f"Nulls In Validation {np.any(np.isnan(val_multi))}")
print(f"Nulls In Test {np.any(np.isnan(test_multi))}")

In [ ]:
#ESEMPIO

n_steps = 3
n_horizon = 2
batch_size = 2
shuffle_buffer = 100
multi_var=True 
expand_dims=False

In [ ]:
data = np.arange(30).reshape((-1,2))
data

In [ ]:
window = n_steps + n_horizon

if expand_dims:
    data = tf.expand_dims(data, axis=-1)

ds = tf.data.Dataset.from_tensor_slices(data)
ds = ds.window(window, shift=n_horizon, drop_remainder=True)
ds = ds.flat_map(lambda x : x.batch(window))
ds = ds.map(lambda x : (x[:-n_horizon], x[-n_horizon:, :1]))
ds = ds.batch(batch_size).prefetch(1)

In [ ]:
for Xk,Yk in ds:
    print()
    print("Xk=",Xk)
    print("-"*50)
    print("Yk=",Yk)
    print("X"*100)
    

In [ ]:
def window_dataset(data, n_steps, n_horizon, batch_size, shuffle_buffer, multi_var=False, expand_dims=False):
    """ 
    Create a windowed tensorflow dataset
    """

    #create a window with n steps back plus the size of the prediction length
    window = n_steps + n_horizon
    
    #expand dimensions to 3D to fit with LSTM inputs
    #creat the inital tensor dataset
    if expand_dims:
        data = tf.expand_dims(data, axis=-1)

    ds = tf.data.Dataset.from_tensor_slices(data)
    
    #create the window function shifting the data by the prediction length
    ds = ds.window(window, shift=n_horizon, drop_remainder=True)
    
    #flatten the dataset and batch into the window size
    ds = ds.flat_map(lambda x : x.batch(window))
    ds = ds.shuffle(shuffle_buffer)    
    
    #create the supervised learning problem x and y and batch
    if multi_var:
        ds = ds.map(lambda x : (x[:-n_horizon], x[-n_horizon:, :1]))
    else:
        ds = ds.map(lambda x : (x[:-n_horizon], x[-n_horizon:]))
    
    ds = ds.batch(batch_size).prefetch(32)
    
    return ds


In [ ]:
tf.random.set_seed(42)

n_steps = 72
n_horizon = 24
batch_size = 2
shuffle_buffer = 1000


ds = window_dataset(train_multi, n_steps, n_horizon, batch_size, shuffle_buffer, multi_var=True)

print('Example sample shapes')
for idx,(x,y) in enumerate(ds):
    print("x = ", x.numpy().shape)
    print("y = ", y.numpy().shape)
    break

In [ ]:
plt.plot(x[0,:,0])
plt.plot(np.arange(72,72+24),y[0])

In [ ]:
print('Example sample shapes')
for idx,(x,y) in enumerate(ds):

    plt.figure(figsize=(26,3))
    plt.plot(x[0,:,0])
    plt.plot(np.arange(72,72+24),y[0])
    plt.grid()

    if idx==10:
        break



In [ ]:
BATCH_SIZE = 256

In [ ]:
def build_dataset(train_fraq=0.8, 
                  n_steps=24*30, 
                  n_horizon=24, 
                  batch_size=BATCH_SIZE, 
                  shuffle_buffer=24*20, 
                  expand_dims=False, 
                  multi_var=False,
                  test_len=365*24,
                  ):
    """
    If multi variate then first column is always the column from which the target is contstructed.
    """
    
    tf.random.set_seed(23)
    
    if multi_var:
        data = load_data(col=['time', 'price actual', 'total load actual'])
        hours, day, months = make_datetime_features(data.time)
        data = pd.concat([data.drop(['time'], axis=1), hours, day, months], axis=1)
    else:
        data = load_data(col=['price actual'])
    
    data = clean_data(data)
    
    y_scaler = MinMaxScaler((-1,1))
    scaler   = MinMaxScaler((-1,1))
    
    y_scaler.fit(data.iloc[:,0:1])
    data = scaler.fit_transform(data)
    
    train_data, val_data, test_data = split_data(data, train_fraq=train_fraq, test_len=test_len)
    
    train_ds = window_dataset(train_data, n_steps, n_horizon, batch_size, shuffle_buffer, multi_var=multi_var, expand_dims=expand_dims)
    val_ds   = window_dataset(val_data, n_steps,  n_horizon,  batch_size, shuffle_buffer, multi_var=multi_var, expand_dims=expand_dims)
    test_ds  = window_dataset(test_data, n_steps, n_horizon,  batch_size, shuffle_buffer, multi_var=multi_var, expand_dims=expand_dims)
    
    print(f"Prediction lookback (n_steps): {n_steps}")
    print(f"Prediction horizon (n_horizon): {n_horizon}")
    print(f"Batch Size: {batch_size}")
    print("Datasets:")
    print(train_ds.element_spec)
    
    return train_ds, val_ds, test_ds, y_scaler


In [ ]:

train_ds, val_ds, test_ds, y_scaler = build_dataset(multi_var=True)

In [ ]:
def get_params(multivar=False):
    lr = 1e-4
    n_steps=24*30
    n_horizon=24
    if multivar:
        n_features=5
    else:
        n_features=1
        
    return n_steps, n_horizon, n_features, lr

def cfg_model_run(model, history, test_ds, val_ds, y_scaler):
    return {"model": model, "history" : history, "test_ds": test_ds, "val_ds" : val_ds, "y_scaler" : y_scaler}


def run_model(model_name, model_func, model_configs, epochs):
    
    n_steps, n_horizon, n_features, lr = get_params(multivar=True)
    train_ds, val_ds, test_ds, y_scaler = build_dataset(n_steps=n_steps, n_horizon=n_horizon, multi_var=True)

    model = model_func(n_steps, n_horizon, n_features, lr=lr)

    model_hist = model.fit(train_ds, validation_data=val_ds, epochs=epochs)

    model_configs[model_name] = cfg_model_run(model, model_hist, test_ds, val_ds, y_scaler)
    return test_ds

In [ ]:
def cnn_model(n_steps, n_horizon, n_features, lr=3e-4):
    
    tf.keras.backend.clear_session()
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(16, kernel_size=3, activation='elu', input_shape=(n_steps,n_features)),
        tf.keras.layers.MaxPooling1D(2),
        tf.keras.layers.Conv1D(16, kernel_size=6, activation='elu'),
        tf.keras.layers.MaxPooling1D(3),
        tf.keras.layers.Conv1D(16, kernel_size=6, activation='elu'),
        tf.keras.layers.MaxPooling1D(4),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(128),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(n_horizon)
    ], name="CNN")
    
    loss= tf.keras.losses.Huber()
    model.compile(loss='mae', optimizer='adam', metrics=['mae'])
    
    return model

cnn = cnn_model(*get_params(multivar=True))
cnn.summary()

In [ ]:
plot_model(cnn, 
           show_shapes=True,
           show_layer_names=True,           
           to_file='model.png')

In [ ]:
def lstm_cnn_skip_model(n_steps, n_horizon, n_features, lr):
    
    tf.keras.backend.clear_session()
   
    inputs = tf.keras.layers.Input(shape=(n_steps,n_features), name='main')
    
    #Primo Ramo
    conv1 = tf.keras.layers.Conv1D(64, kernel_size=6, activation='relu')(inputs)
    max_pool_1 = tf.keras.layers.MaxPooling1D(2)(conv1)
    conv2 = tf.keras.layers.Conv1D(64, kernel_size=3, activation='elu')(max_pool_1)
    max_pool_2 = tf.keras.layers.MaxPooling1D(2)(conv2)
    lstm_1 = tf.keras.layers.LSTM(64, activation='relu', return_sequences=True)(max_pool_2)
    lstm_2 = tf.keras.layers.LSTM(64, activation='relu', return_sequences=False)(lstm_1)
    flatten = tf.keras.layers.Flatten()(lstm_2)
    
    #Secondo ramo: skip
    skip_flatten = tf.keras.layers.Flatten()(inputs)
    dense_skip = tf.keras.layers.Dense(64, activation='relu')(skip_flatten)

    #Concateno i due rami
    concat = tf.keras.layers.Concatenate(axis=-1)([flatten, dense_skip])
    drop_1 = tf.keras.layers.Dropout(0.5)(concat)
    dense_1 = tf.keras.layers.Dense(128, activation='relu')(drop_1)
    drop_2 = tf.keras.layers.Dropout(0.5)(dense_1)
    output = tf.keras.layers.Dense(n_horizon)(drop_2)
    
    model = tf.keras.Model(inputs=inputs, outputs=output, name='lstm_skip')
    
    loss = tf.keras.losses.Huber()
    
    model.compile(loss='mae', optimizer='adam', metrics=['mae'])
    
    return model

lstm_skip = lstm_cnn_skip_model(*get_params(multivar=True))
lstm_skip.summary()

In [ ]:
plot_model(lstm_skip, 
           show_shapes=True,
           show_layer_names=True,           
           to_file='model.png')

In [ ]:
model_configs=dict()
run_model("cnn", cnn_model, model_configs, epochs=600)
run_model("lstm_skip", lstm_cnn_skip_model, model_configs, epochs=600)

In [ ]:
legend = list()

fig, axs = plt.subplots(1, 2, figsize=(25,5))

def plot_graphs(metric, val, ax, upper):
    ax.plot(val['history'].history[metric])
    ax.plot(val['history'].history[f'val_{metric}'])
    ax.set_title(key)
    ax.legend([metric, f"val_{metric}"])
    ax.set_xlabel('epochs')
    ax.set_ylabel(metric)
    # ax.set_ylim([0, upper])
    
for (key, val), ax in zip(model_configs.items(), axs.flatten()):
    plot_graphs('loss', val, ax, 0.2)
print("Loss Curves")

In [ ]:
print("MAE Curves")
fig, axs = plt.subplots(1, 2, figsize=(25,5))
for (key, val), ax in zip(model_configs.items(), axs.flatten()):
    plot_graphs('mae', val, ax, 0.6)

In [ ]:
y_scaler = model_configs['cnn']['y_scaler']

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(24, 12))
days = BATCH_SIZE

vline = np.linspace(0, days*24, days+1)

for (key, val), ax in zip(model_configs.items(), axs):

    test = val['test_ds']

    xbatch, ybatch = iter(test).get_next()
    preds = val['model'].predict(xbatch)

    ybatch = ybatch.numpy()[:days].reshape(-1,1)
    preds  = preds[:days].reshape(-1,1)
    
    ybatch = y_scaler.inverse_transform(ybatch)
    preds = y_scaler.inverse_transform(preds)
    
    rmse = mean_squared_error(ybatch,preds,squared=False)
    print('RMSE:',rmse)

    ax.plot(ybatch)
    ax.plot(preds)
    ax.set_xlim(0,days*24)
    ax.set_title(key + '  =>  RMSE=' + str(rmse))
    ax.vlines(vline, ymin=0, ymax=1, linestyle='dotted', transform = ax.get_xaxis_transform())
    ax.legend(["Actual", "Predicted"])

plt.xlabel("Hours Cumulative")
print('First Two Weeks of Predictions')

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(24, 12))
days = 7*6

vline = np.linspace(0, days*24, days+1)

for (key, val), ax in zip(model_configs.items(), axs):

    test = val['test_ds']

    xbatch, ybatch = iter(test).get_next()
    preds = val['model'].predict(xbatch)

    ybatch = ybatch.numpy()[:days].reshape(-1,1)
    preds  = preds[:days].reshape(-1,1)
    
    ybatch = y_scaler.inverse_transform(ybatch)
    preds = y_scaler.inverse_transform(preds)
    
    rmse = mean_squared_error(ybatch,preds,squared=False)
    print('RMSE:',rmse)

    ax.plot(ybatch)
    ax.plot(preds)
    ax.set_xlim(0,days*24)
    ax.set_title(key + '  =>  RMSE=' + str(rmse))
    ax.vlines(vline, ymin=0, ymax=1, linestyle='dotted', transform = ax.get_xaxis_transform())
    ax.legend(["Actual", "Predicted"])

plt.xlabel("Hours Cumulative")
print('First Two Weeks of Predictions')